<a href="https://colab.research.google.com/github/takatakamanbou/MVA/blob/2022/catPCA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 主成分分析の応用: 猫顔画像の次元圧縮，再構成，生成



## はじめに

主成分分析の応用例を紹介します．猫の顔画像を次元圧縮したりさらにほげほげ（ほげほげが何かは↓で説明してます）したりする実験をやりましょう．



### 猫画像の主成分分析

ここで使う猫顔画像はすべてグレイスケール画像（いわゆる白黒画像）で，$D = 64\times 64 = 4096$ 個の画素から成ります．
このような画像の画素値を並べた $D\times 1$ 行列を $\mathbf{x}$ と表し，$\mathbf{x}$ の平均を $\bar{\mathbf{x}}$ と表すことにします．

また，このデータの分散共分散行列の固有値を $\lambda_1 \geq \lambda_2 \geq \ldots \geq \lambda_D$ として，それぞれに対応する固有ベクトルを $\mathbf{u}_1, \mathbf{u}_2, \ldots, \mathbf{u}_D$ とします．
固有値の大きい方の $H$ 本の固有ベクトルを並べて作った行列を

$$
U_H = \begin{pmatrix}
\mathbf{u}_1 & \mathbf{u}_2 & \cdots & \mathbf{u}_H
\end{pmatrix}
$$

とします．$U_H$ は $D\times H$ 行列です．

このとき，

$$
\mathbf{y} = U_H^{\top}(\mathbf{x} - \bar{\mathbf{x}})\qquad (1)
$$

という変換によって，$D$ 次元の画像 $\mathbf{x}$ が $H \leq D$ 次元の情報 $\mathbf{y}$ に変換されます．


### 猫画像の次元削減と再構成

上記は，授業で説明したことを具体的な猫画像の例で説明しているだけですが，この主成分分析による次元削減の話には実は続きがあります．
式$(1)$の変換によって得られた $\mathbf{y}$ をさらに次のように変換すると，次元数が $D$ に戻ります．この $\mathbf{z}$ のことを，ここでは $\mathbf{x}$ の「再構成」と呼ぶことにします．

$$
\mathbf{z} = U_H\mathbf{y} + \bar{\mathbf{x}} \qquad (2)
$$

$\mathbf{y}$ は $\mathbf{x}$ のもつ情報を $H$ 次元で表しており，$\mathbf{z}$ はそれを $D$ 次元に戻していますので，再構成 $\mathbf{z}$ は $\mathbf{x}$ がもっていた情報をすべて保持しているわけではありません．
しかし，再構成 $\mathbf{z}$ は，元のデータ $\mathbf{x}$ の良い近似になっています（ここではこれくらいのざっくりした説明にとどめておきます．「機械学習I,II」の授業でこの辺を扱うかも）．
特に，$H = D$ の場合，つまり次元を削減しない場合，式 $(1),(2)$ で得られる再構成 $\mathbf{z}$ は完全に $\mathbf{x}$ に一致します．


このようなデータの次元削減と再構成の組み合わせは，コンピュータで扱うデータの量を削減する「データ圧縮」のために使うことができます．
例えば，上記の猫画像 $\mathbf{x}$ ひとつは $D = 4096$ 個の画素値から成ります．これを $H = 100$ 次元の $\mathbf{y}$ に変換してから再構成した $\mathbf{z}$ が元画像に十分近かったとしてみましょう．この場合，あるコンピュータから別のコンピュータへ猫顔画像を送りたいならば，送信側が式$(1)$の計算をして $\mathbf{y}$ を送信し，受信側が式$(2)$の計算をして $\mathbf{z}$ を得ることにすれば，送信するデータの量を $\frac{100}{4096}$ にすることができます．

実際のデータ圧縮はこれほど単純ではありませんが，画像や音声の圧縮の技術（JPEGとか）もこのような話の先にあります（ここでは説明していない他の情報の科学・技術がいろいろ出てきます．コンピュータの仕組みの基礎とかアルゴリズムの話とかフーリエ変換等の数学とか）．


### 猫画像の生成

上記の説明では，元のデータ $\mathbf{x}$ から $\mathbf{y}$ を求め，そこから再構成 $\mathbf{z}$ を得ていました．しかし，$\mathbf{y}$ に適当な数値を与えて式$(2)$ の計算をすれば，画像 $\mathbf{z}$ を得ることができます．このようにすることで，存在しない猫顔画像を作り出す（生成）することもできます．

## 準備

In [ ]:
# 必要なパッケージのインポート
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

次のセルではあとで使う関数を定義しています．このセルを実行しただけでは何も起こりませんが，あらかじめ実行して関数を定義しておかないと後のセルが動きません．

In [ ]:
#####  データの最初の nx x ny 枚を画像化
#
def mosaicImage(dat, nx, ny, nrow=64, ncol=64, gap=4):

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width), dtype = int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            if iy*nx+ix >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol] = dat[iy*nx+ix, :].reshape((nrow, ncol))
            
    return img

画像データを入手して眺めてみましょう．**この実験で使う猫顔画像は，他の目的に使ってはいけません**．

In [ ]:
# takataka のウェブサイトから猫画像データを入手
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/PIP/cat131.npz

import os
path = 'cat131.npz'
if os.path.exists(path):
    cat = np.load(path)['cat131']
    print(cat.shape)  # 131枚，ひとつの画像は 64 x 64 = 4096 画素のグレイスケール画像
else:
    print(f'ファイル {path} の読み込みに失敗したようです．再実行してみてください')

In [ ]:
print('### 猫画像 ###')
print(f'データ数 x 次元数 = {cat.shape[0]} x {cat.shape[1]}')

# 最初の 16 枚を表示 
img = mosaicImage(cat, 4, 4)
plt.figure(figsize=(8, 8))
plt.axis('off')
plt.imshow(img, cmap = 'gray')
plt.show()  

## 実験1 猫顔画像の主成分分析

(1) 平均 $\mathbf{0}$ のデータ行列を作ります．ついでに平均を可視化してみます．


In [ ]:
# 平均 0 のデータ行列を作成
Xm = np.mean(cat, axis=0)
X = cat - Xm
N, D = X.shape
print(X.shape)

# 平均を画像として表示
plt.axis('off')
plt.imshow(Xm.reshape(64, 64), cmap = 'gray')
plt.show()  

(2) データの分散共分散行列の固有値と固有ベクトルを求めます．

In [ ]:
# X の分散共分散行列の固有値と固有ベクトルを求める
_, sval, Vt = np.linalg.svd(X, full_matrices=False)
eval = sval**2/N
print('固有値:', eval)
U = Vt

# 固有ベクトルを可視化
nx, ny = 5, 2
tmp = np.zeros((nx*ny, D))
for h in range(nx*ny):
    tmp[h, :] = U[h, :]
absmax = np.max(np.abs(tmp))
tmp = tmp/absmax*127 + 128
img = mosaicImage(tmp, nx, ny)
plt.figure(figsize=(20, 4))
plt.axis('off')
plt.imshow(img, cmap='gray')
plt.show()  

上記の画像は，固有ベクトルのうち固有値の大きい方から10個を左上から右に向かって並べたものです．固有ベクトルの値には正も負もあるので，正の値が白，0 が灰色，負の値が黒になるようにしてあります．

1つ目の固有ベクトルは，額の部分が白っぽく，鼻筋から顎の下辺りが黒くなっています．したがって，額の部分が平均より明るく，鼻筋から顎の下辺りが平均より暗くなっている猫画像に対して大きな値となります．2つ目以降も，それぞれ異なる特徴を捉えていることがわかります．

ちなみに，↑のセルでの `np.linalg.svd` の使い方は，授業の notebook のやり方とは違っています．興味があれば takataka に聞いてね（omake05も関係してます）．

(3) 一つの画像を選んで，その変換後の特徴量を出力させてみましょう．`n` の値を変えると，他の猫も選べます．

In [ ]:
n = 12 # データの番号． 0 から 130 まで

# 元の画像を表示
plt.axis('off')
plt.imshow(cat[n, :].reshape(64, 64), cmap = 'gray')
plt.show()

# 変換して得られる特徴量のうち10個を出力
y = X[n, :] @ U[:10, :].T
print(y)

例えば，顎下の部分に注目すると，そこが黒っぽい猫は1つ目の成分が正で，白っぽい猫は負になっているはずです．猫と固有ベクトルを見比べていろいろ調べてみましょう．

## 実験2 次元削減と再構成

次のセルを実行すると，主成分分析で得られた変換行列を利用して，猫画像を $H$ 次元に次元削減してから再構成します．

In [ ]:
#@title 猫画像を次元削減してから再構成
H =  0#@param {type: "number"}
assert 0 <= H and H <= D

Z = np.zeros_like(X) + Xm
if H > 0:
    Y = X @ U[:H, :].T
    Z += Y @ U[:H, :]

nx, ny = 2, 10
tmp = np.zeros((nx*ny, D))
for n in range(10):
    tmp[2*n,   :] = cat[n, :]
    tmp[2*n+1, :] = Z[n, :]
img = mosaicImage(tmp, nx, ny)

print(f'{H}次元での再構成')
plt.figure(figsize=(3, 15))
plt.axis('off')
plt.imshow(img, cmap = 'gray')
plt.show()

上記の画像では，学習データのうち10匹の猫について，左に元画像，右に再構成画像が並んでます．`H` の値をいろいろ変えてみましょう．

## 実験3 手動で猫顔画像を生成させてみよう


次のセルでは，$(y_1, y_2, y_3, y_4, y_5)$ の値を自由に選んで，それを再構成して猫画像を生成することができます．適当に値を選んでみよう．

In [ ]:
#@title 猫画像を生成
y1 =  1000 #@param {type: "slider", min:-2000, max:2000}
y2 = -2000 #@param {type: "slider", min:-2000, max:2000}
y3 = -1200 #@param {type: "slider", min:-2000, max:2000}
y4 =   200 #@param {type: "slider", min:-2000, max:2000}
y5 = -1000 #@param {type: "slider", min:-2000, max:2000}

y = np.array([y1, y2, y3, y4, y5])
z = y @ U[:5, :] + Xm

# 生成した画像を表示
plt.axis('off')
plt.imshow(z.reshape(64, 64), cmap='gray')
plt.show()

## 実験4 無限に猫顔画像を生成させてみよう

次のようにして，猫顔画像を無限に生成させてみましょう．

1. $H$ 次元の $\mathbf{y}$ の値が多次元正規分布に従うと仮定して，分散共分散行列を推定します.
    - 平均は $\mathbf{0}$ のはず
    - 主成分スコアの分散共分散行列は，固有値が対角に並んだ対角行列のはず
1. 推定したパラメータを持つ多次元正規分布に従う疑似乱数を発生させます．
1. その値を $\mathbf{y}$ として式 $(2)$ を計算することによって，猫顔画像 $\mathbf{z}$ を生成します．

In [ ]:
# 次元削減したデータに正規分布を当てはめる

H = 32  # 変換後の次元数

# 正規分布のパラメータの推定．PCAしたデータの平均は 0 で分散共分散行列は固有値の並んだ対角行列
mu = np.zeros(H)
cov = np.diag(eval[:H])
print(eval[:H])

次のセルを実行すると，猫顔画像を25枚生成します．
実行するたびに乱数の値が変わるので，何度も実行してみましょう．

In [ ]:
# 平均 mu，共分散 cov の多次元正規分布に従う乱数を生成
from numpy.random import default_rng
rng = default_rng()
Yt = rng.multivariate_normal(mean=mu, cov=cov, size=25)
print(Yt.shape)

# Yt を再構成
Z = Yt @ U[:H, :] + Xm

# 画素値の範囲 [0, 255] をはみ出した値の処理
ZZ = Z.reshape(-1)
ZZ[ZZ < 0] = 0
ZZ[ZZ > 255] = 255
print(np.min(Z), np.max(Z))

# 可視化
img = mosaicImage(Z, 5, 5)
plt.figure(figsize=(8, 8))
plt.axis('off')
plt.imshow(img, cmap='gray')
plt.show()

この実験では，

- 使っているデータの数が少ない
- 生成モデルがただの多次元正規分布なので単純すぎる

ため，生成される画像の品質は高くありません．近年急速に発達している画像生成の方法は，ここでやっているような計算を基礎的な考え方としつつ，最先端の機械学習手法を用いたものとなっています．